## Подготавливаем данные

В качестве исходного материала для чат-бота будем использовать ролики с YouTube. Для того, чтобы с ними работать, установим библиотеку `PyTube`.

In [1]:
%pip install pytube

Defaulting to user installation because normal site-packages is not writeable


Возможно, прежде чем выполнять следующий код, вам потребуется перезапустить Kernel.

Посмотрим, как можно обратиться к ролику на YouTube по ссылке:

In [2]:
from pytube import YouTube
yt = YouTube("https://www.youtube.com/watch?v=7mWPlXvPG7A")
yt.streams

[<Stream: itag="17" mime_type="video/3gpp" res="144p" fps="6fps" vcodec="mp4v.20.3" acodec="mp4a.40.2" progressive="True" type="video">, <Stream: itag="18" mime_type="video/mp4" res="360p" fps="25fps" vcodec="avc1.42001E" acodec="mp4a.40.2" progressive="True" type="video">, <Stream: itag="22" mime_type="video/mp4" res="720p" fps="25fps" vcodec="avc1.64001F" acodec="mp4a.40.2" progressive="True" type="video">, <Stream: itag="137" mime_type="video/mp4" res="1080p" fps="25fps" vcodec="avc1.640028" progressive="False" type="video">, <Stream: itag="248" mime_type="video/webm" res="1080p" fps="25fps" vcodec="vp9" progressive="False" type="video">, <Stream: itag="136" mime_type="video/mp4" res="720p" fps="25fps" vcodec="avc1.4d401f" progressive="False" type="video">, <Stream: itag="247" mime_type="video/webm" res="720p" fps="25fps" vcodec="vp9" progressive="False" type="video">, <Stream: itag="135" mime_type="video/mp4" res="480p" fps="25fps" vcodec="avc1.4d401e" progressive="False" type="vid

In [3]:
yt.streams.filter(mime_type="audio/webm").first().download(output_path="../audio",filename="1.opus")

'/home/jupyter/work/resources/VideoQABot/../audio/1.opus'

In [5]:
import glob
import librosa
import soundfile as sf

target_sr = 8000
for fn in glob.glob("../audio/*.opus"):
    print(f"Processing {fn}")
    au,sr = librosa.load(fn,sr=target_sr)
    sf.write(fn.replace('.opus','.ogg'),au,target_sr,format='ogg',subtype='opus')
    

Processing ../audio/1.opus


<ipython-input-2-eb729a1953cc>:8: UserWarning: PySoundFile failed. Trying audioread instead.
  au,sr = librosa.load(fn,sr=target_sr)
/usr/local/lib/python3.10/dist-packages/librosa/core/audio.py:184: FutureWarning: librosa.core.audio.__audioread_load
	Deprecated as of librosa version 0.10.0.
	It will be removed in librosa version 1.0.
  y, sr_native = __audioread_load(path, offset, duration, dtype)


Копируем аудио-файлы в хранилище S3, подключенное к DataSphere, чтобы можно было использовать их для распознавания речи.

In [6]:
!cp ../audio/*.ogg /home/jupyter/mnt/s3/mclass/audio

In [7]:
#!c1.4
api_id = os.environ['api_id']
api_key = os.environ['api_key']

In [8]:
#!c1.4
import requests

audio_file = "https://storage.yandexcloud.net/mclass/audio/1.ogg"

j = {
    "config": {
        "specification": {
            "languageCode": "ru-RU",
        }
    },
    "audio": {
        "uri": audio_file
    }
}

res = requests.post("https://transcribe.api.cloud.yandex.net/speech/stt/v2/longRunningRecognize",
                    json = j,
                    headers = { "Authorization" : f"Api-Key {api_key}" })


In [16]:
res = res.json()
id = res.json()['id']
print(res)

In [19]:
import time 

while True:
    res = requests.get(f"https://operation.api.cloud.yandex.net/operations/{id}",headers = { "Authorization" : f"Api-Key {api_key}" })
    res = res.json()
    if res['done']:
        break
    print("Waiting")
    time.sleep(10)

In [22]:
res['response']['chunks'][0]

{'alternatives': [{'words': [{'startTime': '41.180s',
     'endTime': '41.879s',
     'word': 'встречайте',
     'confidence': 1},
    {'startTime': '42.759s',
     'endTime': '43.400s',
     'word': 'руководитель',
     'confidence': 1},
    {'startTime': '43.439s',
     'endTime': '44.239s',
     'word': 'платформы',
     'confidence': 1},
    {'startTime': '44.410s',
     'endTime': '44.819s',
     'word': 'яндекс',
     'confidence': 1},
    {'startTime': '44.899s',
     'endTime': '45.370s',
     'word': 'клауд',
     'confidence': 1},
    {'startTime': '46.080s',
     'endTime': '46.600s',
     'word': 'алексей',
     'confidence': 1},
    {'startTime': '46.660s',
     'endTime': '47.339s',
     'word': 'башкеев',
     'confidence': 1}],
   'text': 'Встречайте руководитель платформы яндекс клауд алексей башкеев',
   'confidence': 1}],
 'channelTag': '1'}

In [26]:
' '.join([x['alternatives'][0]['text'] for x in res['response']['chunks']])

'Встречайте руководитель платформы яндекс клауд алексей башкеев Добрый день Я рад вас приветствовать на 4 ежегодной конференции яндекс кейл Посвященный облачным технологиям Облачные технологии являются трендом во всем мире Российский рынок не исключение мы продолжаем расти Трехзначными темпами как яндекс клауд опережая рост облачного рынка Это происходит благодаря вам нашим клиентам нашим пользователям вас уже больше 23000 это огромная цифра За эти 4 года наша платформа существенно изменилась Изменилось и то как вы ее используете Говоря о изменениях на базовом инфраструктурном слое стоит сказать что еще 4 года назад основным сценарием использования облачной платформы были просто виртуальные машины и какая то небольшая автоматизация поверх них сейчас появились 2 новых тренда это контейнерные стейки и прежде всего cubernetics и Без серверного подхода который находит все больше и больше областей применения во всех сферах бизнеса Говоря о контейнерах стоит в 1 очередь сказать о cubernetics

In [27]:
with open('/home/jupyter/mnt/s3/mclass/text/1.txt','w',encoding='utf-8') as f:
    f.write(' '.join([x['alternatives'][0]['text'] for x in res['response']['chunks']]))